In [7]:
import cv2
import numpy as np

In [8]:
def pre_process(path):
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    scale_factor = 700 / max(image.shape[:2])
    image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor)
    gray = cv2.resize(gray, (0, 0), fx=scale_factor, fy=scale_factor)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV, 11, 2)
    return (image, thresh, scale_factor)
    
def show(img,time = 3):
    cv2.imshow("Image", img)
    cv2.waitKey(int(time * 1000))
    cv2.destroyAllWindows()

def edge(image, thresh, scale_factor):
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    coin_contours = []
    for cnt in contours:
        perimeter = cv2.arcLength(cnt, True)
        area = cv2.contourArea(cnt)
        if perimeter:
            circularity = 4 * np.pi * (area / (perimeter ** 2))
            if 0.7 < circularity < 1.2 and area > 2000 * (scale_factor ** 2):  
                coin_contours.append(cnt)
    cv2.drawContours(image, coin_contours, -1, (0, 255, 0), 2)
    show(image,3)
    return coin_contours

    
def contour_segmentation(image, coin_contours):
    segmented_coins = []
    for cnt in coin_contours:
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        center = (int(x), int(y))
        radius = int(radius)
        mask = np.zeros_like(image, dtype=np.uint8)
        cv2.circle(mask, center, radius, (255, 255, 255), -1)
        coin_segment = cv2.bitwise_and(image, mask)
        x1, y1, x2, y2 = center[0] - radius, center[1] - radius, center[0] + radius, center[1] + radius
        coin_segment = coin_segment[y1:y2, x1:x2]
        segmented_coins.append(coin_segment)
    for i, coin in enumerate(segmented_coins):
        show(coin,2)
    return segmented_coins

    
def count_coin(coin_contours, segmented_coins):
    return (len(coin_contours), len(segmented_coins))

In [50]:

import cv2
import numpy as np

def segment_coins_knn(path):
    # Load the image
    image = cv2.imread(path)
    image_copy = image.copy()

    # SCALE-INVARIANT RESIZING
    scale_factor = 700 / max(image.shape[:2])  
    image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor)

    # Convert image to LAB color space and extract L-channel
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    L_channel = lab[:, :, 0]  # Extract only the Luminance channel

    # Apply Gaussian Blur to smoothen noise
    blurred = cv2.GaussianBlur(L_channel, (5, 5), 0)

    # Reshape for K-Means Clustering
    pixel_values = blurred.reshape((-1, 1))  # Single-channel input
    pixel_values = np.float32(pixel_values)

    # Apply K-Means Clustering
    k = 3  # Number of clusters (background + coins)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centers = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Convert labels to mask (segment the darkest region)
    labels = labels.reshape(image.shape[:2])
    mask = np.uint8(labels == np.argmin(centers)) * 255  # Coins are usually darker

    # Apply Otsu’s thresholding for cleaner segmentation
    _, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations to refine mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)  # Fill small holes
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)  # Remove noise

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    coin_images = []
    show(mask)
    # Find contours of the segmented coins
    # contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # coin_contours = []
    # for cnt in contours:
    #     perimeter = cv2.arcLength(cnt, True)
    #     area = cv2.contourArea(cnt)
    #     if perimeter:
    #         circularity = 4 * np.pi * (area / (perimeter ** 2))
    #         # if 0.7 < circularity < 1.2 and area > 2000 * (scale_factor ** 2):  
    #         coin_contours.append(cnt)
    # cv2.drawContours(image, coin_contours, -1, (0, 255, 0), 2)

    # # Show segmented result
    # show(image)

    # return coin_contours  # Return segmented coin images

# Run function on image
path = "coins3.jpg"
coins = segment_coins_knn(path)

In [38]:
def main(path):
    image, thresh,scale_factor = pre_process(path)
    coin_contours = edge(image, thresh, scale_factor)
    segmented_coins = contour_segmentation(image, coin_contours)
    print(count_coin(coin_contours, segmented_coins))

In [39]:
# main("coins.jpg")
# main("coins1.jpg")
# main("coins2.jpg")
# main("coins3.jpg")
# main("coins4.jpg")
# main("coins5.jpg")
# main("coins6.jpg")
# main("coins7.jpg")
# # main("coins8.jpg")
# main("coins9.jpg")
# main("coins10.jpg")


In [41]:
# segment_coins_watershed("coins.jpg")
# segment_coins_watershed("coins1.jpg")
# segment_coins_watershed("coins2.jpg")
segment_coins_knn("coins3.jpg")
# segment_coins_watershed("coins4.jpg")
# segment_coins_watershed("coins5.jpg")
# segment_coins_watershed("coins6.jpg")
# segment_coins_watershed("coins7.jpg")
# # segment_coins_watershed("coins8.jpg")
# segment_coins_watershed("coins9.jpg")
# segment_coins_watershed("coins10.jpg")


error: OpenCV(4.10.0) /croot/opencv-suite_1738943342777/work/modules/core/src/arithm.cpp:230: error: (-215:Assertion failed) (mtype == CV_8U || mtype == CV_8S) && _mask.sameSize(*psrc1) in function 'binary_op'
